# Creating the dataset

In [ ]:
import pandas as pd
import requests
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor

In [ ]:
import os
os.chdir("/Users/jacobjjli/Library/CloudStorage/OneDrive-UniversityofToronto/Documents/School/0-6 Grad Apps/Predoc Applications/openalex")

In [ ]:
# Import

filename = "openalex_names_full_study_df10_0_100000_slim.csv"
path = "/content/drive/MyDrive/2YP/02_temp/full_study_slim/"
in_df = pd.read_csv(path+filename)

In [ ]:
# add headers so that code is added into OpenAlex's "polite" pool
headers = {'mailto':'jacobjj.li@mail.utoronto.ca'}

# Defining Functions

In [ ]:
class Bucket:
    '''
    A token bucket to rate limit API requests.
    '''
    def __init__(self, max_tokens, refill_rate):
        """
        Initialize a new token bucket with a maximum number of tokens
        max_tokens which regenerate after refill_rate seconds. tokens represents
        the number of tokens currently in the bucket. current_time represents
        the current time and is used in refill behaviour.

        token_checkout_times is a list of the last ten times that a token was
        checked out and is initialized to current_time repeated 10 times. In the
        course of operating API calls, token_checkout_times will be ordered in
        an increasing order. That is, the earliest times (smallest values)
        appear first. Old checkout times are removed from the beginning of the
        list (index 0) and new checkout times are added to the end of the list
        (index 9).  As such, token_checkout_times should almost always have
        length 10, but it may be less than 10 if API calls take a long time
        to come back (i.e., new API calls are made before some API calls
        are returned). token_checkout_times will be constantly updated so that
        refill can check if enough time has elapsed before refilling the token
        bucket.

        An API request can only be made if tokens is greater than 1.
        """

        self.max_tokens = max_tokens
        self.tokens = max_tokens
        self.current_time = time.perf_counter()
        self.token_checkout_times = [self.current_time] * self.max_tokens
        self.refill_rate = refill_rate

    def refill(self):
        """
        Increment the token count by 1 if the current time is at least
        refill_rate after the 10th last token was checked out (the first item
        in token_checkout_times).

        >>> bucket = Bucket(10, 2)
        >>> bucket.get_token()
        >>> bucket.tokens == 9
        True
        >>> bucket.refill()
        >>> bucket.tokens == 9
        True
        >>> time.sleep(bucket.refill_rate)
        >>> bucket.refill()
        >>> bucket.tokens == 10
        True
        """
        self.current_time = time.perf_counter()
        while len(self.token_checkout_times) == 0:
            time.sleep(0.01)
            self.current_time = time.perf_counter()
        if self.current_time - self.token_checkout_times[0] >= self.refill_rate:
            self.tokens += 1

    def get_token(self):
        """
        If tokens < 1, then continue calling refill to attempt refilling the
        token bucket. This prevents the API call from being made.

        If tokens >= 1, then "check out" a token by decreasing tokens by 1 and
        removing the first (earliest) item of token_checkout_times. After the
        API call is succesfully made, functions should call update_checkout_time
        to add the time of the successful call to token_checkout_times.

        Together with update_checkout_times, get_token can be thought of as
        "pushing out" the first item of token_checkout_times from the start of
        the list and adding the current time to the end of the list.

        """
        while self.tokens < 1:
            self.refill()
            time.sleep(0.1)

        self.token_checkout_times.pop(0)
        self.tokens -= 1

    def update_checkout_time(self):
        """
        Add the current time to the end of token_checkout_times.
        update_checkout_times should only be called after get_token to ensure
        that token_checkout_times has 10 elements in the long run.
        """
        self.token_checkout_times = self.token_checkout_times + [time.perf_counter()]

In [ ]:
WORKS_COLUMNS = ['pub_total', 'pub_id', 'pub_doi', 'pub_title',
                  'put_year', 'pub_type', 'pub_cited_by_url,', 'pub_cites',
                  'pub_journal_id', 'pub_journal_name',
                  'pub_keywords', 'authorship_position', 'pub_facility_id',
                  'pub_facility_display_name', 'pub_facility_ror',
                  'pub_facility_country','pub_facility_type', 'timestamp_download']

In [ ]:
def search_works(in_df: pd.DataFrame):
    """
    Return a dataframe (out_df) containing bibliographic information from
    OpenAlex about works produced by the authors listed in in_df. in_df must
    contain at least the following two columns:
     - diss_uid: a unique string identifier for each author assigned by the
       author name data source (ex. 1995c3b3df3cd2)
     - works_api_url: a URL string containing the link to the OpenAlex directory
       containing the author's works
       (ex. https://api.openalex.org/works?filter=author.id:A2001663675
       corresponds to the author name data source identifier 1995c3b3df3cd2)

    out_df is a dataframe with columns WORKS_COLUMNS, each corresponding to some
    piece of bibliographic information in the OpenAlex system. Each row will
    usually correspond to one publication. More speficifically, if an author
    in in_df has:
     - NO publications listed on OpenAlex, there will be one row in out_df with
       the author's diss_uid and works_api_url. This row will have a 0 entry in
       the pub_total column and null entries in all other columns.
     - ONE publication listed, there will be one row in out_df with the author's
       diss_uid and works_api_url. This row will have a 1 entry in the pub_total
       column and entries reflecting its bibliographic information in all other
       columns.
     - N publications listed (with N > 1), there will be N rows in out_df each
       containing the same diss_uid and works_api_url. Each row will correspond
       to a different publication, and hence the entries in other columns (ex.
       pub_doi or pub_year) will differ between rows.

    """
    # create output dataframe
    out_df = pd.DataFrame(columns=['works_api_url', 'diss_uid'] + WORKS_COLUMNS)

    # create a token bucket with 10 max tokens and 1 second between refills
    bucket = Bucket(10, 1)

    # extract URLs to send to OpenAlex API from in_df
    df_url_clean = clean_url_list(in_df)
    works_url_list = list(df_url_clean['works_api_url'])
    diss_uids = list(df_url_clean['diss_uid'])

    # execute requests from in_df in parallel
    with ThreadPoolExecutor(max_workers=12) as pool:
        futures = [pool.submit(query_API_works, url, diss_uid, 200, bucket)
                   for (url, diss_uid) in zip(works_url_list, diss_uids)]

        # add the result of each hit to out_df
        for future in futures:
            out_df = pd.concat((out_df, future.result()))

    return out_df

In [ ]:
def query_API_works(works_url: str, diss_uid: str, per_page: int,
                    token_bucket: Bucket):
    """
    Return a dataframe (df) containing selected variables corresponding to the
    works returned by an OpenAlex search query for works_url. diss_uid is not
    used in the API call but is reassigned to df (which contains the search
    results) after the API call to help identify which author the results
    correspond to. per_page corresponds to the OpenAlex per_page parameter and
    controls how many results are displayed per page. token_bucket is
    shared between all parallel processes of query_API_works and is used to
    rate limit API requests to adhere to OpenAlex's 10 requests per second limit.
    """
    print(f'Starting works query for {works_url}.')
    df = pd.DataFrame(columns=WORKS_COLUMNS) # create output df
    (pub_total, response_json) = issue_query(works_url, '*', per_page,
                                             token_bucket) # send API request

    # if 503 error, change per_page to 25 and resend query
    while response_json == '<Response [503]>':
        per_page = 25
        print('503 error encountered. Results per page has been ' +
              'changed to 25 (default value: 200)')
        (pub_total, response_json) = issue_query(works_url, '*', per_page,
                                                 token_bucket)

    # record general information about search results
    pub_total = response_json['meta']['count']
    results = response_json['results']

    # if the author has no works, create an entry with no bibliographic info
    if pub_total == 0:
        df = df.append({'pub_total': 0}, ignore_index=True)

    # if all results are on one page, iterate over each result and extract data
    elif pub_total <= per_page:
        results = response_json['results']
        for result in results:
            variables = extract_variables_works(works_url, result)
            variables = [pub_total] + variables
            df.loc[len(df)] = variables

    # if more than one page of results, iterate over pages and then over results
    else:
        for i in range(0, pub_total, per_page):
            results = response_json['results']
            # extract the cursor to navigate to the next page
            next_cursor = response_json['meta']['next_cursor']
            for result in results:
                variables = extract_variables_works(works_url, result)
                variables = [pub_total] + variables
                df.loc[len(df)] = variables
            # issue query for the next page of results
            response_json = issue_query(works_url, next_cursor, per_page,
                                        token_bucket)[1]

            # if 503 error, change per_page to 25 and resend query
            while response_json == '<Response [503]>':
                per_page = 25
                print('503 error encountered. Results per page has been ' +
                      'changed to 25 (default value: 200)')
                (pub_total, response_json) = issue_query(works_url, next_cursor,
                                                         per_page, token_bucket)

    # add the identification parameters (diss_url, diss_uid) back into dataframe
    df = add_uid_url(df, works_url, diss_uid)
    print(f'Finished query for {works_url}.')

    return df

In [ ]:
def add_uid_url(df: pd.DataFrame, works_url: str, diss_uid: str):
    df['works_api_url'] = works_url
    df['diss_uid'] = diss_uid

    return df

In [ ]:
def issue_query(works_url: str, cursor: str, per_page: int,
                token_bucket: Bucket):
    search_url = (f'{works_url}&per_page={per_page}&cursor={cursor}')
    # attempt to acquire token before launching API call
    token_bucket.get_token()
    oa_response = requests.get(search_url, headers=headers)
    # update the list of checkout times once API call is successful
    token_bucket.update_checkout_time()
    response_code = str(oa_response)

    flag = False
    while response_code != '<Response [200]>':
        # save the error code to print later
        error_code = response_code
        print(response_code + search_url)
        if response_code == '<Response [503]>':
            # pass control back to query_API_works
            return (0, response_code)
        else:
            token_bucket.get_token()
        oa_response = requests.get(search_url, headers=headers)
        token_bucket.update_checkout_time()
        response_code = str(oa_response)
        flag = True

    if flag:
        print('RESOLVED ' + error_code + search_url)

    response_json = oa_response.json()
    # get total number of hits
    pub_total = response_json['meta']['count']

    return (pub_total, response_json)

In [ ]:
def clean_url_list(in_df: pd.DataFrame):
    """
    There are some author searches which didn't lead to a works url. We want to remove those from our search
    """
    # in_df['works_api_url'].replace('', np.nan, inplace=True)
    in_df.dropna(subset=['works_api_url'], inplace=True)
    out_df = in_df[['works_api_url', 'diss_uid']]


    return out_df

In [ ]:
def extract_variables_works(works_url: str, result: list):
    """
    Return a list (variables) containing variables of interest corresponding to
    a specific OpenAlex API search result (result).
    """
    variables = []

    # pull selected variables from each result and add it to variables
    # if no such variable exists, leave empty string

    #results
    ### pub_id
    try:
        variables.append(result['id'])
    except:
        variables.append('')
    ### pub_doi
    try:
        variables.append(result['doi'])
    except:
        variables.append('')
    ### pub_title
    try:
        variables.append(result['title'])
    except:
        variables.append('')
    ### pub_year
    try:
        variables.append(result['publication_year'])
    except:
        variables.append('')
    ### pub_type
    try:
        variables.append(result['type'])
    except:
        variables.append('')
    ### pub_cited_by_url
    try:
        variables.append(result['cited_by_api_url'])
    except:
        variables.append('')
    ### pub_cites
    try:
        variables.append(result['cited_by_count'])
    except:
        variables.append('')


    # results / host venue
    ## pub_journal_id:
    try:
        variables.append(result['host_venue']['id'])
    except:
        variables.append('')
    ## pub_journal_name:
    try:
        variables.append(result['host_venue']['display_name'])
    except:
        variables.append('')

    # results / concepts:
    ### pub_keywords
    concepts = result['concepts']
    keywords = []
    for concept in concepts:
        try:
            keywords.append(concept['display_name'])
        except:
            keywords.append('')

    variables.append(keywords)

    # results / authorship
    ### authorship_position
    authorship = result['authorships']

    cnt=0
    stop_loop = 0
    for author in authorship:
        author_id_round = works_url.split("filter=author.id:")
        author_id_round = author_id_round[1]

        try:
            author_id_authorship = author['author']['id']
        except:
            author_id_authorship = "openalex.org/"

        author_id_authorship = author_id_authorship.split("openalex.org/")
        author_id_authorship = author_id_authorship[1]

        if (author_id_round == author_id_authorship):
            if stop_loop !=1:
                try:
                    authorship_position = author['author_position']
                    variables.append(author['author_position'])
                except:
                    variables.append('')

                ### results / authorship / author

                ###results / authorships / institutions
                # institutions = authorship['institutions']
                # print('institutions',institutions)

                try:
                    pub_facility_id = author['institutions'][cnt]['id']
                    variables.append(author['institutions'][cnt]['id'])
                except:
                    #print('no institutions info') # --> it says there is no institutions information coded
                    variables.append('')

                try:
                    pub_facility_display_name = author['institutions'][cnt]['display_name']
                    variables.append(author['institutions'][cnt]['display_name'])
                except:
                  variables.append('')

                try:
                    pub_facility_ror = author['institutions'][cnt]['ror']
                    variables.append(author['institutions'][cnt]['ror'])
                except:
                    variables.append('')

                try:
                    pub_facility_country = author['institutions'][cnt]['country_code']
                    variables.append(author['institutions'][cnt]['country_code'])
                except:
                    variables.append('')

                try:
                    pub_facility_type = author['institutions'][cnt]['type']
                    variables.append(author['institutions'][cnt]['type'])
                except:
                    variables.append('')

                cnt = cnt+1
                stop_loop = 1

    if stop_loop ==0:
        variables.append('')
        variables.append('')
        variables.append('')
        variables.append('')
        variables.append('')
        variables.append('')


    # Timestamp Download:
    import datetime
    now = datetime.datetime.now()
    variables.append(now.strftime("%Y_%m_%d"))

    return variables

# Doctests

In [ ]:
import doctest
doctest.testmod()


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.8/doctest.py", line 1487, in run
    sys.settrace(save_trace)



TestResults(failed=0, attempted=8)

# Testing with 503 Error

In [ ]:
# test with 1 URL with Joseph Stiglitz's works (1496 results)
df_one = df1[1:2]
df_one['works_api_url'] = 'https://api.openalex.org/works?filter=author.id:A2585687038'
df_one['diss_name'] = '503 Test'

<ipython-input-31-e333b29d2cc2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one['works_api_url'] = 'https://api.openalex.org/works?filter=author.id:A2585687038'
<ipython-input-31-e333b29d2cc2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one['diss_name'] = '503 Test'


In [ ]:
df_one

,diss_name,diss_uid,works_api_url
1,503 Test,1995c3b3df3cd2,https://api.openalex.org/works?filter=author.i...


In [ ]:
%%time
df_one_results = search_works(df_one)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
CPU times: user 12 s, sys: 85.4 ms, total: 12.1 s
Wall time: 18.6 s


In [ ]:
# test with 11 URLs with Joseph Stiglitz's works (1496 results each)
df_eleven = pd.concat([df_one]*11, ignore_index=True)

In [ ]:
%%time
search_works(df_eleven)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]><Response [200]>

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

,works_api_url,diss_uid,pub_total,pub_id,pub_doi,pub_title,put_year,pub_type,"pub_cited_by_url,",pub_cites,pub_journal_id,pub_journal_name,pub_keywords,authorship_position,pub_facility_id,pub_facility_display_name,pub_facility_ror,pub_facility_country,pub_facility_type,timestamp_download
0,https://api.openalex.org/works?filter=author.i...,1995c3b3df3cd2,1963,https://openalex.org/W1546523058,https://doi.org/10.7916/d8v12ft1,Credit Rationing in Markets with Imperfect Inf...,1981,posted-content,https://api.openalex.org/works?filter=cites:W1...,5175,https://openalex.org/V23254222,The American Economic Review,"[Credit rationing, Collateral, Economics, Loan...",first,,,,,,2023_02_20
1,https://api.openalex.org/works?filter=author.i...,1995c3b3df3cd2,1963,https://openalex.org/W2121034459,https://doi.org/10.2307/1885326,Equilibrium in Competitive Insurance Markets: ...,1976,journal-article,https://api.openalex.org/works?filter=cites:W2...,3865,https://openalex.org/S203860005,Quarterly Journal of Economics,"[Economics, Imperfect, Perfect information, In...",last,None,Stanford University and All Souls College,None,None,None,2023_02_20
2,https://api.openalex.org/works?filter=author.i...,1995c3b3df3cd2,1963,https://openalex.org/W1485676192,https://doi.org/10.22004/ag.econ.268957,Monopolistic competition and optimum product d...,1977,journal-article,https://api.openalex.org/works?filter=cites:W1...,3743,https://openalex.org/V23254222,The American Economic Review,"[Numéraire, Monopolistic competition, Economic...",last,,,,,,2023_02_20
3,https://api.openalex.org/works?filter=author.i...,1995c3b3df3cd2,1963,https://openalex.org/W2032899120,https://doi.org/10.4135/9781452206905.n316,Globalization and Its Discontents,2016,reference-entry,https://api.openalex.org/works?filter=cites:W2...,3280,https://openalex.org/V201928713,Review of Radical Political Economics,"[Globalization, International trade, Political...",first,,,,,,2023_02_20
4,https://api.openalex.org/works?filter=author.i...,1995c3b3df3cd2,1963,https://openalex.org/W1995118028,https://doi.org/10.1016/0022-0531(70)90038-4,Increasing risk: I. A definition,1970,journal-article,https://api.openalex.org/works?filter=cites:W1...,3194,https://openalex.org/S149131268,Journal of Economic Theory,[Economics],last,https://openalex.org/I80134939,Canisius College,https://ror.org/01056ge55,US,education,2023_02_20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,https://api.openalex.org/works?filter=author.i...,1995c3b3df3cd2,1963,https://openalex.org/W91994301,None,Introduction to Research in Taxation,1978,posted-content,https://api.openalex.org/works?filter=cites:W9...,0,https://openalex.org/S3121261024,Research Papers in Economics,[Economics],last,,,,,,2023_02_20
1959,https://api.openalex.org/works?filter=author.i...,1995c3b3df3cd2,1963,https://openalex.org/W936860898,None,Has the New Millennium Repealed the Old Econom...,2004,None,https://api.openalex.org/works?filter=cites:W9...,0,None,None,"[Law, Political science, Law and economics, Ec...",first,,,,,,2023_02_20
1960,https://api.openalex.org/works?filter=author.i...,1995c3b3df3cd2,1963,https://openalex.org/W962665774,https://doi.org/10.1017/cbo9780511615207.005,How finance differs,2003,book-chapter,https://api.openalex.org/works?filter=cites:W9...,0,https://openalex.org/S4306462995,Cambridge University Press eBooks,"[Business, Finance]",first,,,,,,2023_02_20
1961,https://api.openalex.org/works?filter=author.i...,1995c3b3df3cd2,1963,https://openalex.org/W976781670,https://doi.org/10.1017/cbo9780511615207.019,The new paradigm and the “new economy”,2003,book-chapter,https://api.openalex.org/works?filter=cites:W9...,0,https://openalex.org/S4306462995,Cambridge University Press eBooks,"[History, Economics]",first,,,,,,2023_02_20


# Def Run Functions Multiple

In [ ]:
def run_functions_multiple(in_df, start, stop, queries_per_file,
                           queries_per_call, master_path):
    '''
    Call run_functions to generate multiple files with query results from in_df.
    Each output file will have name beginning with master_path with the index
    number of the first query appended to the end (ex. some_path/some_file_120).
    master_path should NOT have a file extension (.csv) at the end.

    Start querying at start and stop querying at stop. queries_per_file decides
    how many queries will have their results written to a specific file, whereas
    queries_per_call is passed to run_functions and determines how many queries
    are sent to search_names at once. queries_per_file should be greater than
    queries_per_call.
    '''
    if queries_per_file < queries_per_call:
        print(f'Please set queries_per_file to be greater than queries_per_call'
              f'Function execution terminated.')
        return

    for i in range(start, stop, queries_per_file):
        print(f'Starting on query {str(start)}.')
        run_functions(in_df, i, i+queries_per_file, queries_per_call,
                      master_path+'openalex_works_full_study_'+str(i)+'_'+str(i+queries_per_file)+'.csv')


In [ ]:
def create_slim_df(in_path: str, out_path: str, columns_to_keep: list):
    in_df = pd.read_csv(in_path)
    in_df = in_df[columns_to_keep]
    in_df.to_csv(out_path, index=False, header=True, encoding='utf8')
    return None

In [ ]:
def run_functions(in_df: pd.DataFrame, start: int, stop: int, step: int,
                  path: str):
    '''
    Write the results of search_names to path. Take in_df and perform search
    queries on the rows from start to stop. Starts a new search using search_names
    every step rows to increase performance.

    Creates a new file to write output. If file already exists, prompts the user
    on whether they want to append new output to existing file. Append is useful
    when search_names stops unexpectedly, leaving an incomplete file. The user
    would then restart querying at the point where the querying stopped and
    would choose to append to an existing file.
    '''
    # generate one set of headers
    df_headers = pd.DataFrame(columns=['diss_name', 'diss_uid']+WORKS_COLUMNS)

    # try to create the file if it doesn't already exist
    try:
        df_headers.to_csv(path, index=False, header=True, mode='x')
        print(f'NEW FILE CREATED at {path}. \n')
    except FileExistsError:
        print(f'FILE ALREADY EXISTS at {path}. Append instead? \n')
        # request user input to prevent accidental appending
        response = input('Y/N: ')
        if response.upper() != 'Y':
           return

    # create slice of in_df
    in_df = in_df[start:stop]
    # execute search_names function for subslices of in_df determined by step
    for i in range(0,len(in_df), step):
        print(f'====INITIATING queries {str(i)} to {str(i+step)}====')
        df_temp = in_df[i:i+step]
        df_results = search_works(df_temp)
        print(f'\nFINISHED queries {str(i)} to {str(i+step)}.', end=' ')
        print(f'WRITING to {path}...', end = ' ')
        df_results.to_csv(path, index=False, header=False, mode='a',
                          encoding='utf8')
        print('SUCCESSFULLY wrote to file.')

    print(f'\nSUCCESS in querying items {str(start)} to {str(stop)}. (Query end)')

    # create "slim" df for next step of OA code
    slim_columns = ['pub_facility_id']
    slim_path = path[:-4] + '_slim' + '.csv'
    create_slim_df(path, slim_path , slim_columns)
    print(f'Created slim df at {slim_path} containing columns {str(slim_columns)}.\n')

    return None

# Pulling Data

In [ ]:
'''
def run_functions_multiple(in_df, start, stop, queries_per_file,
                           queries_per_call, master_path):

Call run_functions to generate multiple files with query results from in_df.
Each output file will have name beginning with master_path with the index
number of the first query appended to the end (ex. some_path/some_file_120).
master_path should NOT have a file extension (.csv) at the end.

Start querying at start and stop querying at stop. queries_per_file decides
how many queries will have their results written to a specific file, whereas
queries_per_call is passed to run_functions and determines how many queries
are sent to search_names at once. queries_per_file should be greater than
queries_per_call.
'''

# example code that sends requests for df1[0:1000], creates two files of 500
# authors each, and calls search_works 10 times in chunks of 100 names
run_functions_multiple(df1, 255000, 580903, 5000, 200, 'drive/MyDrive/2YP/02_temp/full_study_works/df1/df1')

Starting on query 255000.
NEW FILE CREATED at drive/MyDrive/2YP/02_temp/full_study_works/df1/df1openalex_works_full_study_255000_260000.csv. 

====INITIATING queries 0 to 200====
Starting works query for https://api.openalex.org/works?filter=author.id:A3147503291.
Starting works query for https://api.openalex.org/works?filter=author.id:A3147607936.
Starting works query for https://api.openalex.org/works?filter=author.id:A3147613782.
Starting works query for https://api.openalex.org/works?filter=author.id:A3147683889.
Starting works query for https://api.openalex.org/works?filter=author.id:A3147815736.
Starting works query for https://api.openalex.org/works?filter=author.id:A3147906946.
Starting works query for https://api.openalex.org/works?filter=author.id:A3147919889.
Starting works query for https://api.openalex.org/works?filter=author.id:A3148213934.
Starting works query for https://api.openalex.org/works?filter=author.id:A3149061597.
Starting works query for https://api.openalex.o

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Streaming output truncated to the last 5000 lines.
Finished query for https://api.openalex.org/works?filter=author.id:A4311996406.
Starting works query for https://api.openalex.org/works?filter=author.id:A4313082533.
Finished query for https://api.openalex.org/works?filter=author.id:A4312484619.
Starting works query for https://api.openalex.org/works?filter=author.id:A4313082535.
Finished query for https://api.openalex.org/works?filter=author.id:A4311971719.
Starting works query for https://api.openalex.org/works?filter=author.id:A4313082605.
Finished query for https://api.openalex.org/works?filter=author.id:A4312499487.
Starting works query for https://api.openalex.org/works?filter=author.id:A4313083082.
Finished query for https://api.openalex.org/works?filter=author.id:A4312506373.
Starting works query for https://api.openalex.org/works?filter=author.id:A4313089115.
Finished query for https://api.openalex.org/works?filter=author.id:A4312511264.
Starting works query for https://api.op

KeyboardInterrupt: ignored